In [78]:
import psycopg2


# Establish the connection
connection = psycopg2.connect(
    host='jobjack-tech-assessments-do-user-3965150-0.b.db.ondigitalocean.com',
    user='doadmin',
    password='AVNS_rbiTyQ_BS7wAYbXjFhV',
    database='tech_assess_thato',
    connect_timeout=180,
    port=25060
)

# Connection is established
print("Connection successful!")

# Perform database operations

# Close the connection
cursor = connection.cursor()

query = '''SELECT *
    FROM jack_location_density;'''
cursor.execute(query)

#Fetch all rows at once
rows = cursor.fetchall()
connection.close()

Connection successful!


In [79]:
import pandas as pd
pd.DataFrame(rows)

,0,1,2,3
0,2,2018,Eastern Cape,1
1,2,2018,Western Cape,15
2,2,2018,Unidentified,60
3,3,2018,Gauteng,1
4,3,2018,Western Cape,55
...,...,...,...,...
636,5,2023,Limpopo,3176
637,5,2023,Mpumalanga,1907
638,5,2023,Northern Cape,702
639,5,2023,North West,1970


In [81]:
jack_location_density = pd.DataFrame(rows)

In [82]:
jack_location_density.columns = ['month','year','province','sign_ups']

In [83]:
jack_location_density.head()

,month,year,province,sign_ups
0,2,2018,Eastern Cape,1
1,2,2018,Western Cape,15
2,2,2018,Unidentified,60
3,3,2018,Gauteng,1
4,3,2018,Western Cape,55


In [97]:
jack_location_density = jack_location_density[jack_location_density['province'].isin(list(jack_location_density['province'].unique())[:11])]

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [98]:
# Load the preprocessed data
data = jack_location_density

In [139]:
model.feature_names_in_

array(['province_Eastern Cape', 'province_Free State', 'province_Gauteng',
       'province_KwaZulu-Natal', 'province_Kwazulu Natal',
       'province_Limpopo', 'province_Mpumalanga', 'province_North West',
       'province_Northern Cape', 'province_Unidentified',
       'province_Western Cape', 'month', 'year'], dtype=object)

In [149]:
# Split the data into features (X) and target variable (y)
X = data.drop("sign_ups", axis=1)
y = data["sign_ups"]

In [100]:
# Perform one-hot encoding on the "province" column
ohe = OneHotEncoder(sparse=False)
X_encoded = ohe.fit_transform(X[["province"]])
encoded_columns = ohe.get_feature_names_out(["province"])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_columns)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [101]:
# Normalize the remaining features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.drop("province", axis=1))
X_scaled_df = pd.DataFrame(X_scaled, columns=X.drop("province", axis=1).columns)

In [102]:
# Concatenate the encoded and scaled features
X_processed = pd.concat([X_encoded_df, X_scaled_df], axis=1)

In [103]:
X_processed.head()

,province_Eastern Cape,province_Free State,province_Gauteng,province_KwaZulu-Natal,province_Kwazulu Natal,province_Limpopo,province_Mpumalanga,province_North West,province_Northern Cape,province_Unidentified,province_Western Cape,month,year
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.090909,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.090909,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.181818,0.0


In [104]:
# Split the processed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [128]:
# Train the Gradient Boosting Regression model
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor()

In [129]:
# Make predictions on the testing set
y_pred = model.predict(X_test)

In [130]:
X_test.head()

,province_Eastern Cape,province_Free State,province_Gauteng,province_KwaZulu-Natal,province_Kwazulu Natal,province_Limpopo,province_Mpumalanga,province_North West,province_Northern Cape,province_Unidentified,province_Western Cape,month,year
355,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.454545,0.6
407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.909091,0.6
90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.090909,0.2
402,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.6
268,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.727273,0.4


In [109]:
# Evaluate the model's performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)

Mean Absolute Error: 769.0403722441513
Root Mean Squared Error: 1291.2864041790463


In [122]:
# Predict sign-ups for 2025
new_data = pd.DataFrame({
    "year": [2025] * len(data['province'].unique()),
    "province": list(data['province'].unique())
})

In [132]:
data

,month,year,province,sign_ups
date,,,,
2018-02-01,2,2018,Eastern Cape,1
2018-02-01,2,2018,Western Cape,15
2018-02-01,2,2018,Unidentified,60
2018-03-01,3,2018,Gauteng,1
2018-03-01,3,2018,Western Cape,55
...,...,...,...,...
2023-05-01,5,2023,Limpopo,3176
2023-05-01,5,2023,Mpumalanga,1907
2023-05-01,5,2023,Northern Cape,702


In [134]:
new_data

,year,province,month
0,2025,Eastern Cape,1
1,2025,Western Cape,1
2,2025,Unidentified,1
3,2025,Gauteng,1
4,2025,Kwazulu Natal,1
5,2025,KwaZulu-Natal,1
6,2025,Mpumalanga,1
7,2025,North West,1
8,2025,Limpopo,1
9,2025,Free State,1


In [148]:
# Perform one-hot encoding on the new data
new_data_encoded = ohe.transform(new_data[["province"]])
new_data_encoded_df = pd.DataFrame(new_data_encoded, columns=encoded_columns)

# Normalize the remaining features in the new data
new_data_scaled = scaler.fit_transform(new_data.drop("province", axis=1))
new_data_scaled_df = pd.DataFrame(new_data_scaled, columns=new_data.drop("province", axis=1).columns)

# Concatenate the encoded and scaled features for the new data
new_data_processed = pd.concat([new_data_encoded_df, new_data_scaled_df], axis=1)
new_data_processed = new_data_processed[X_processed.columns]
# Predict sign-ups for 2025 using the trained model
predicted_sign_ups = model.predict(new_data_processed)

predictions = pd.DataFrame({
    "month": new_data["month"],
    "year": new_data["year"],
    "province": new_data["province"],
    "predicted_sign_ups": predicted_sign_ups
})

predictions["predicted_sign_ups"] = predictions["predicted_sign_ups"].apply(lambda x: abs(x) if x < 0 else x)
predictions["predicted_sign_ups"] = predictions["predicted_sign_ups"].astype(int)
print(predictions)


    month  year       province  predicted_sign_ups
0       1  2025   Eastern Cape                 248
1       1  2025   Western Cape                 962
2       1  2025   Unidentified                 721
3       1  2025        Gauteng                 266
4       1  2025  Kwazulu Natal                  87
5       1  2025  KwaZulu-Natal                 295
6       1  2025     Mpumalanga                 483
7       1  2025     North West                 316
8       1  2025        Limpopo                 290
9       1  2025     Free State                 265
10      1  2025  Northern Cape                 293
